In [13]:
'''
Importing relevant libraries for project.
'''
import pandas as pd
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from selenium.webdriver.support import expected_conditions as EC

In [14]:
'''
Importing CSV file valid postcodes as a list.
'''
POA_2016 = open('VIC_postcodes_batch2.csv')
valid_postcodes = csv.reader(POA_2016)

for row in valid_postcodes:
    valid_postcodes = row

In [15]:
valid_postcodes

['3463',
 '3464',
 '3465',
 '3467',
 '3468',
 '3469',
 '3472',
 '3475',
 '3477',
 '3478',
 '3480',
 '3482',
 '3483',
 '3485',
 '3487',
 '3488',
 '3489',
 '3490',
 '3491',
 '3494',
 '3496',
 '3498',
 '3500',
 '3501',
 '3505',
 '3506',
 '3507',
 '3509',
 '3512',
 '3515',
 '3516',
 '3517',
 '3518',
 '3520',
 '3521',
 '3522',
 '3523',
 '3525',
 '3527',
 '3529',
 '3530',
 '3531',
 '3533',
 '3537',
 '3540',
 '3542',
 '3544',
 '3546',
 '3549',
 '3550',
 '3551',
 '3555',
 '3556',
 '3557',
 '3558',
 '3559',
 '3561',
 '3562',
 '3563',
 '3564',
 '3565',
 '3566',
 '3567',
 '3568',
 '3570',
 '3571',
 '3572',
 '3573',
 '3575',
 '3576',
 '3579',
 '3580',
 '3581',
 '3583',
 '3584',
 '3585',
 '3586',
 '3588',
 '3589',
 '3590',
 '3591',
 '3594',
 '3595',
 '3596',
 '3597',
 '3599',
 '3607',
 '3608',
 '3610',
 '3612',
 '3614',
 '3616',
 '3617',
 '3618',
 '3620',
 '3621',
 '3622',
 '3623',
 '3624',
 '3629',
 '3630',
 '3631',
 '3633',
 '3634',
 '3635',
 '3636',
 '3637',
 '3638',
 '3639',
 '3640',
 '3641',
 

In [10]:
'''
Creating a counter for number of iterations and an empty master dictionary for scraped IGA store data to be appended to later on.
'''
store_counter = 0
IGA_stores = { }

'''
Selenium set up and navigation to IGA 'Store locator' website.
'''
browser = webdriver.Chrome()
browser.get('https://www.iga.com.au/stores/#view=storelocator')

'''
The following block of code is for Selenium to simulate a user navigating through the IGA website and inputting postcodes into the search box.

Selenium will:
Navigate to the search box, clear text (if there is any) from the search box, input a postcode, wait for the suggestions box to load, press the down arrow
key and then enter key to select the first suggestion of the dropdown list.
      
This code is within a for loop, to iterate through the list of valid Australian postcodes.
'''
for postcode in valid_postcodes:
    
    time.sleep(5)
    
    search_box = browser.find_element_by_id("sf-location-search")
    
    search_box.clear()
    search_box.send_keys(postcode)
    search_box.click()
    
    time.sleep(5)

    search_box.send_keys(Keys.ARROW_DOWN)
    search_box.send_keys(Keys.RETURN)
    
    time.sleep(5)
    
    """
    Selenium will now scrape the store name and store address for each store in the search results. These resulting text is then cleaned
    before being stored in lists.
    """
    store_name = [ ]
    store_address = [ ]

    for store in browser.find_elements_by_xpath('.//span[@class = "sf-storename"]'):
        store_name.append(store.text)

    for address in browser.find_elements_by_xpath('.//p[@class = "sf-storeaddress"]'):
        store_address.append(address.text)

    cleaned_store_address_list = [ ]

    for store in store_address:
        cleaned_store_address_list.append(store.replace('\n' , ','))
    
    """
    Iterate through the two lists of store name and store address and add them to the master dictionary that we created earlier.
    """
    for i in range(len(store_name)):
        IGA_stores.update( {store_name[i] : cleaned_store_address_list[i] } )
    
    print(f"Stores for postcode {postcode} have been added to the master dictionary")
    
    store_counter += 1
    
    percentage_progress = float(store_counter)/len(valid_postcodes)*100 
    print(f"Progress: {round(percentage_progress,1)}%")
    
    """
    Now that all search results for this postcode have been extracted and stored, find and click on the change location button on the website, ready for 
    the next postcode to be input into the search box.
    """
    change_location = browser.find_element_by_id("sf-location-change")
    change_location.click()

Stores for postcode 3000 have been added to the master dictionary
Progress: 0.1%
Stores for postcode 3002 have been added to the master dictionary
Progress: 0.3%
Stores for postcode 3003 have been added to the master dictionary
Progress: 0.4%
Stores for postcode 3004 have been added to the master dictionary
Progress: 0.6%
Stores for postcode 3005 have been added to the master dictionary
Progress: 0.7%
Stores for postcode 3006 have been added to the master dictionary
Progress: 0.9%
Stores for postcode 3008 have been added to the master dictionary
Progress: 1.0%
Stores for postcode 3010 have been added to the master dictionary
Progress: 1.1%
Stores for postcode 3011 have been added to the master dictionary
Progress: 1.3%
Stores for postcode 3012 have been added to the master dictionary
Progress: 1.4%
Stores for postcode 3013 have been added to the master dictionary
Progress: 1.6%
Stores for postcode 3015 have been added to the master dictionary
Progress: 1.7%
Stores for postcode 3016 hav

Stores for postcode 3115 have been added to the master dictionary
Progress: 14.6%
Stores for postcode 3116 have been added to the master dictionary
Progress: 14.8%
Stores for postcode 3121 have been added to the master dictionary
Progress: 14.9%
Stores for postcode 3122 have been added to the master dictionary
Progress: 15.1%
Stores for postcode 3123 have been added to the master dictionary
Progress: 15.2%
Stores for postcode 3124 have been added to the master dictionary
Progress: 15.4%
Stores for postcode 3125 have been added to the master dictionary
Progress: 15.5%
Stores for postcode 3126 have been added to the master dictionary
Progress: 15.6%
Stores for postcode 3127 have been added to the master dictionary
Progress: 15.8%
Stores for postcode 3128 have been added to the master dictionary
Progress: 15.9%
Stores for postcode 3129 have been added to the master dictionary
Progress: 16.1%
Stores for postcode 3130 have been added to the master dictionary
Progress: 16.2%
Stores for postc

Stores for postcode 3226 have been added to the master dictionary
Progress: 29.0%
Stores for postcode 3227 have been added to the master dictionary
Progress: 29.1%
Stores for postcode 3228 have been added to the master dictionary
Progress: 29.3%
Stores for postcode 3230 have been added to the master dictionary
Progress: 29.4%
Stores for postcode 3231 have been added to the master dictionary
Progress: 29.6%
Stores for postcode 3232 have been added to the master dictionary
Progress: 29.7%
Stores for postcode 3233 have been added to the master dictionary
Progress: 29.8%
Stores for postcode 3234 have been added to the master dictionary
Progress: 30.0%
Stores for postcode 3235 have been added to the master dictionary
Progress: 30.1%
Stores for postcode 3236 have been added to the master dictionary
Progress: 30.3%
Stores for postcode 3237 have been added to the master dictionary
Progress: 30.4%
Stores for postcode 3238 have been added to the master dictionary
Progress: 30.6%
Stores for postc

Stores for postcode 3378 have been added to the master dictionary
Progress: 43.3%
Stores for postcode 3379 have been added to the master dictionary
Progress: 43.5%
Stores for postcode 3380 have been added to the master dictionary
Progress: 43.6%
Stores for postcode 3381 have been added to the master dictionary
Progress: 43.8%
Stores for postcode 3384 have been added to the master dictionary
Progress: 43.9%
Stores for postcode 3385 have been added to the master dictionary
Progress: 44.0%
Stores for postcode 3387 have been added to the master dictionary
Progress: 44.2%
Stores for postcode 3388 have been added to the master dictionary
Progress: 44.3%
Stores for postcode 3390 have been added to the master dictionary
Progress: 44.5%
Stores for postcode 3391 have been added to the master dictionary
Progress: 44.6%
Stores for postcode 3392 have been added to the master dictionary
Progress: 44.8%
Stores for postcode 3393 have been added to the master dictionary
Progress: 44.9%
Stores for postc

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="sf-location-search"]"}
  (Session info: chrome=80.0.3987.122)


In [11]:
IGA_stores

{'IGA Xpress Melbourne (Lonsdale Street)': '35-41 Lonsdale Street,Melbourne,VIC 3000',
 'Exhibition Street IGA X-press': '333 Exhibition Street,Melbourne,VIC 3000',
 'IGA Xpress Melbourne (Flinders Street)': '84 Flinders Street,Melbourne,VIC 3000',
 'IGA X-press Carlton': '103 - 107 Lygon Street,Carlton,VIC 3053',
 'IGA Xpress Melbourne (Collins St)': '470 Collins Street,Melbourne,VIC 3000',
 'IGA East Melbourne': '416-446 Victoria Parade Shop 2,East Melbourne,VIC 3002',
 'IGA Abbotsford': '306-310 Johnston Street,Abbotsford,VIC 3047',
 'IGA Fitzroy': '424 Brunswick Street,Fitzroy,VIC 3065',
 'SUPA IGA North Melbourne': '20 - 26 Errol Street,North Melbourne,VIC 3051',
 'IGA Xpress North Melbourne': '75 Flemington Road,North Melbourne,VIC 3051',
 'IGA X-press 370 Queen Street': '370 Queen Street,Melbourne,VIC 3000',
 'IGA Melrose': '23 Melrose Street,North Melbourne,VIC 3051',
 'IGA Xpress Docklands': '20 Rakaia Way New Quay,Docklands,VIC 3008',
 'IGA Xpress Melbourne (Commercial Rd)': 

In [12]:
'''
Finally, take the fully populated master dictionary and output to a .csv file.
'''
(pd.DataFrame.from_dict(data = IGA_stores, orient = 'index').to_csv('scraped_VIC_IGA_stores_batch2.csv', header=False))